In [22]:
import spacy
import pandas as pd
from collections import Counter

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 10_000_000

In [3]:
df = pd.read_csv(r"../data/processed/top_150_fantasy_reviews_sentiment.csv")
df.head()

,review_id,anime_title,review_url,date,username,user_review_count,is_preliminary,episodes_watched,recommendation,rating,...,review_sentiment,review_nerpos,textblob_polarity,textblob_subjectivity,textblob_label,vader_neg,vader_neu,vader_pos,vader_compound,vader_label
0,503754,Sousou no Frieren,https://myanimelist.net/reviews.php?id=503754,"Oct 13, 2023 8:38 AM",Czekaj,5,True,5/28,Recommended,10,...,with lives so short why do we even bother? to ...,"With lives so short, why do we even bother? To...",0.087698,0.506092,neutral,0.068,0.853,0.079,0.8890,positive
1,519189,Sousou no Frieren,https://myanimelist.net/reviews.php?id=519189,"Mar 22, 2024 12:40 PM",chekkit,25,False,NaN,Recommended,10,...,i feel so catered to it feels like an eternity...,I feel so catered to. It feels like an eternit...,0.188848,0.523496,positive,0.051,0.757,0.192,0.9987,positive
2,519472,Sousou no Frieren,https://myanimelist.net/reviews.php?id=519472,"Mar 24, 2024 2:03 AM",Trikkiez,3,False,NaN,Not Recommended,4,...,style frieren does not have its own unique sty...,Style- Frieren does not have its own unique st...,-0.000063,0.544437,neutral,0.096,0.782,0.122,0.9958,positive
3,512466,Sousou no Frieren,https://myanimelist.net/reviews.php?id=512466,"Jan 12, 2024 11:25 AM",ShabbaRico,12,True,18/28,Not Recommended,5,...,tl dr 5 10 i do not recommend this for anyone ...,"TL;DR: 5/10, I do not recommend this for anyon...",0.074623,0.449968,neutral,0.062,0.823,0.115,0.9991,positive
4,503760,Sousou no Frieren,https://myanimelist.net/reviews.php?id=503760,"Oct 13, 2023 9:10 AM",TheRealist68,16,True,6/28,Mixed Feelings,9,...,through 3 episodes frieren appears to be a uni...,"Through 3 episodes, Frieren appears to be a un...",0.136992,0.483176,positive,0.036,0.818,0.146,0.9981,positive


In [23]:
# Combine all reviews into one string
all_text = " ".join(df["review_nerpos"].dropna().astype(str).tolist())

# Process the large text
doc = nlp(all_text)

# POS counts
pos_counts = Counter([token.pos_ for token in doc])
print("POS Tag Counts:\n", pos_counts)

# NER counts
ner_counts = Counter([ent.label_ for ent in doc.ents])
print("NER Label Counts:\n", ner_counts)


KeyboardInterrupt



In [13]:
text = df.loc[0, "review_nerpos"]  # This should be the version with casing and punctuation preserved
doc = nlp(text)
pos_counts = Counter([token.pos_ for token in doc])
print(pos_counts)


Counter({'NOUN': 112, 'VERB': 88, 'PUNCT': 82, 'PRON': 81, 'ADP': 65, 'DET': 55, 'ADJ': 46, 'AUX': 41, 'ADV': 36, 'PART': 34, 'SCONJ': 21, 'CCONJ': 19, 'PROPN': 9, 'NUM': 5})


In [9]:
for token in doc[:20]:  # First 20 tokens
    print(f"{token.text:15} | POS: {token.pos_:7} | Tag: {token.tag_:5} | Lemma: {token.lemma_}")


With            | POS: SCONJ   | Tag: IN    | Lemma: with
lives           | POS: NOUN    | Tag: NNS   | Lemma: life
so              | POS: ADV     | Tag: RB    | Lemma: so
short           | POS: ADJ     | Tag: JJ    | Lemma: short
,               | POS: PUNCT   | Tag: ,     | Lemma: ,
why             | POS: SCONJ   | Tag: WRB   | Lemma: why
do              | POS: AUX     | Tag: VBP   | Lemma: do
we              | POS: PRON    | Tag: PRP   | Lemma: we
even            | POS: ADV     | Tag: RB    | Lemma: even
bother          | POS: VERB    | Tag: VB    | Lemma: bother
?               | POS: PUNCT   | Tag: .     | Lemma: ?
To              | POS: ADP     | Tag: IN    | Lemma: to
someone         | POS: PRON    | Tag: NN    | Lemma: someone
who             | POS: PRON    | Tag: WP    | Lemma: who
lives           | POS: VERB    | Tag: VBZ   | Lemma: live
for             | POS: ADP     | Tag: IN    | Lemma: for
thousands       | POS: NOUN    | Tag: NNS   | Lemma: thousand
of              | POS

### NER

In [10]:
for ent in doc.ents:
    print(f"{ent.text:30} | Label: {ent.label_}")

thousands of years             | Label: DATE
Frieren                        | Label: ORG
tomorrow                       | Label: DATE
only one                       | Label: CARDINAL
one                            | Label: CARDINAL
the season                     | Label: DATE
Frieren                        | Label: ORG
Himmel                         | Label: ORG
four                           | Label: CARDINAL
Frieren                        | Label: ORG
Sousou no Frieren              | Label: ORG


In [11]:
ner_counts = Counter([ent.label_ for ent in doc.ents])
print("NER counts:\n", ner_counts)

NER counts:
 Counter({'ORG': 5, 'DATE': 3, 'CARDINAL': 3})
